In [53]:
import requests
import pandas as pd
# response = requests.get("http://localhost:8000/create-db")
# print(response)
response = requests.get("http://localhost:8000/get-db")
df = pd.DataFrame(response.json()["data"])
df['createdat'] = pd.to_datetime(df['createdat'], format="ISO8601", utc=True)
df

,createdat,pm10,pm25,temperatur,rel_luftfeuchte,luftdruck
0,2022-07-25 00:01:49.463000+00:00,10.73,9.93,22.98,73.16,99784.28
1,2022-07-25 00:04:39.011000+00:00,10.80,9.80,23.02,71.94,99785.34
2,2022-07-25 00:07:27.816000+00:00,11.70,10.70,22.95,71.77,99781.03
3,2022-07-25 00:10:16.348000+00:00,11.77,10.77,22.95,71.67,99777.47
4,2022-07-25 00:13:04.992000+00:00,11.10,10.20,22.89,72.10,99773.19
...,...,...,...,...,...,...
519032,2025-06-29 19:58:07.217000+00:00,2.60,2.40,28.60,49.85,100442.88
519033,2025-06-29 20:00:55.094000+00:00,2.30,2.10,27.85,50.60,100334.41
519034,2025-06-29 20:03:47.264000+00:00,3.10,2.87,28.26,51.70,100439.84
519035,2025-06-29 20:06:35.134000+00:00,3.37,3.07,28.03,53.49,100444.56


In [57]:
import numpy as np
df = df.replace([np.nan, np.inf, -np.inf], None)
response = requests.get("http://localhost:9000/predict")
forecast = pd.DataFrame(response.json())[["ds","yhat","yhat_lower","yhat_upper"]]
forecast["ds"] = pd.to_datetime(forecast["ds"], format="ISO8601", utc=True)
forecast

,ds,yhat,yhat_lower,yhat_upper
0,2022-07-25 00:00:00+00:00,22.424655,17.488646,27.049758
1,2022-07-25 01:00:00+00:00,22.130565,17.295101,26.881456
2,2022-07-25 02:00:00+00:00,21.804430,16.881478,26.614713
3,2022-07-25 03:00:00+00:00,21.534216,16.722295,26.030053
4,2022-07-25 04:00:00+00:00,21.516394,16.667570,26.501747
...,...,...,...,...
24675,2025-07-06 13:00:00+00:00,29.703234,24.875925,34.343129
24676,2025-07-06 14:00:00+00:00,29.584950,25.370683,34.241752
24677,2025-07-06 15:00:00+00:00,29.139114,24.028494,33.978704
24678,2025-07-06 16:00:00+00:00,28.384003,23.481151,33.115509


In [58]:
merged = pd.merge_asof(
    df,
    forecast,
    left_on="createdat",
    right_on="ds",
    direction="backward"
)
merged

,createdat,pm10,pm25,temperatur,rel_luftfeuchte,luftdruck,ds,yhat,yhat_lower,yhat_upper
0,2022-07-25 00:01:49.463000+00:00,10.73,9.93,22.98,73.16,99784.28,2022-07-25 00:00:00+00:00,22.424655,17.488646,27.049758
1,2022-07-25 00:04:39.011000+00:00,10.8,9.8,23.02,71.94,99785.34,2022-07-25 00:00:00+00:00,22.424655,17.488646,27.049758
2,2022-07-25 00:07:27.816000+00:00,11.7,10.7,22.95,71.77,99781.03,2022-07-25 00:00:00+00:00,22.424655,17.488646,27.049758
3,2022-07-25 00:10:16.348000+00:00,11.77,10.77,22.95,71.67,99777.47,2022-07-25 00:00:00+00:00,22.424655,17.488646,27.049758
4,2022-07-25 00:13:04.992000+00:00,11.1,10.2,22.89,72.1,99773.19,2022-07-25 00:00:00+00:00,22.424655,17.488646,27.049758
...,...,...,...,...,...,...,...,...,...,...
519032,2025-06-29 19:58:07.217000+00:00,2.6,2.4,28.6,49.85,100442.88,2025-06-29 19:00:00+00:00,25.498630,20.805747,30.220009
519033,2025-06-29 20:00:55.094000+00:00,2.3,2.1,27.85,50.6,100334.41,2025-06-29 20:00:00+00:00,24.743760,19.931326,29.398577
519034,2025-06-29 20:03:47.264000+00:00,3.1,2.87,28.26,51.7,100439.84,2025-06-29 20:00:00+00:00,24.743760,19.931326,29.398577
519035,2025-06-29 20:06:35.134000+00:00,3.37,3.07,28.03,53.49,100444.56,2025-06-29 20:00:00+00:00,24.743760,19.931326,29.398577


In [50]:
df.dtypes

createdat          datetime64[ns, UTC]
pm10                           float64
pm25                           float64
temperatur                     float64
rel_luftfeuchte                float64
luftdruck                      float64
dtype: object

In [32]:
import numpy as np
df = df.replace([np.nan, np.inf, -np.inf], None)
response = requests.post("http://localhost:9000/train", json=df[["createdat","temperatur"]].to_dict())
response

<Response [200]>

In [38]:
response.json()

[{'ds': '2022-07-25T00:00:00',
  'yhat': 22.42465514204788,
  'yhat_lower': 17.655616948308086,
  'yhat_upper': 27.311263153655844},
 {'ds': '2022-07-25T01:00:00',
  'yhat': 22.130565240311192,
  'yhat_lower': 17.302184642704816,
  'yhat_upper': 27.10039172410028},
 {'ds': '2022-07-25T02:00:00',
  'yhat': 21.804429888141662,
  'yhat_lower': 16.79714122750126,
  'yhat_upper': 26.231215073464767},
 {'ds': '2022-07-25T03:00:00',
  'yhat': 21.53421550539872,
  'yhat_lower': 16.74741762673111,
  'yhat_upper': 26.158889867264374},
 {'ds': '2022-07-25T04:00:00',
  'yhat': 21.51639397572142,
  'yhat_lower': 16.590422964310196,
  'yhat_upper': 26.489938594653164},
 {'ds': '2022-07-25T05:00:00',
  'yhat': 21.954842341593647,
  'yhat_lower': 17.2984457349492,
  'yhat_upper': 26.934584239192212},
 {'ds': '2022-07-25T06:00:00',
  'yhat': 22.919862048112257,
  'yhat_lower': 18.256313586262756,
  'yhat_upper': 27.661283949503993},
 {'ds': '2022-07-25T07:00:00',
  'yhat': 24.274786940043796,
  'yhat_l

In [50]:
# df["createdat"] = pd.to_datetime(df['createdat'],format='ISO8601').dt.tz_localize(None)
import pandas as pd

# Versuche, alle Werte in datetime zu konvertieren, aber mit errors='coerce' (ungültige werden NaT)
df['createdat'] = pd.to_datetime(df['createdat'],format='ISO8601', utc=True).dt.tz_localize(None)

df

,createdat,pm10,pm25,temperatur,rel_luftfeuchte,luftdruck
0,2022-07-25 00:01:49.463,10.73,9.93,22.98,73.16,99784.28
1,2022-07-25 00:04:39.011,10.80,9.80,23.02,71.94,99785.34
2,2022-07-25 00:07:27.816,11.70,10.70,22.95,71.77,99781.03
3,2022-07-25 00:10:16.348,11.77,10.77,22.95,71.67,99777.47
4,2022-07-25 00:13:04.992,11.10,10.20,22.89,72.10,99773.19
...,...,...,...,...,...,...
518940,2025-06-29 15:32:00.204,8.13,7.53,35.43,32.96,100532.38
518941,2025-06-29 15:34:49.828,4.93,3.30,35.50,33.72,100524.22
518942,2025-06-29 15:37:41.863,8.03,7.23,35.61,33.54,100524.44
518943,2025-06-29 15:40:33.881,3.50,3.30,35.53,32.91,100509.53


In [21]:
import pandas as pd
from prophet import Prophet
import plotly_express as px

# Annahme: df ist dein DataFrame mit 'createdat' und 'pm10'

# Prophet erwartet die Spalten 'ds' und 'y'
df_prophet = df[['createdat', 'temperatur']].copy()
df_prophet['createdat'] = pd.to_datetime(df_prophet['createdat'], format="ISO8601", utc=True).dt.tz_localize(None)
df_prophet["hour"] = df_prophet["createdat"].dt.to_period("h").dt.to_timestamp()
df_prophet = df_prophet.groupby("hour").mean(numeric_only=True).reset_index().rename(columns={"hour":"ds", "temperatur":"y"})
df_prophet

,ds,y
0,2022-07-25 00:00:00,22.598571
1,2022-07-25 01:00:00,21.869524
2,2022-07-25 02:00:00,21.129545
3,2022-07-25 03:00:00,20.382381
4,2022-07-25 04:00:00,20.052381
...,...,...
24507,2025-06-29 13:00:00,36.060000
24508,2025-06-29 14:00:00,35.870476
24509,2025-06-29 15:00:00,35.450952
24510,2025-06-29 16:00:00,34.753810
